In [1]:
# Importing the packages we will need
from __future__ import print_function
from __future__ import division
from IPython.display import display, clear_output, HTML
from ipywidgets import Button, Layout, GridBox, ButtonStyle, HBox, VBox, Label, Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec

import ipywidgets as ipw
import pandas as pd
import math
import numpy as np
import os.path
import glob
import warnings
import platform
import time

#font = {'family' : 'normal',
#        'weight' : 'normal',
#        'size'   : 22}

#plt.rc('font', **font)
plt.rcParams.update({'font.size': 12})

In [2]:
def getvalue(x): 
    return x

# Make user interface

style = {'description_width': '170px'}
w1 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=3.0,
    min=0,
    max=30.0,
    step=0.1,
    description='$a_0$: Max accu in m year$^{-1}$:',
    style=style,
    disabled=False
))
# display(w1)

w2 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.01,
    min=0,
    max=0.01,
    step=0.0001,
    description='$a_1$: Accu slope in year$^{-1}$:',
    style=style,
    disabled=False
))
# display(w2)

w3 = interactive(getvalue, x=ipw.BoundedIntText(
    value=100,
    min=10,
    max=500,
    step=1,
    description='$N$: Number of grid points:',
    style=style,
    disabled=False
))
# display(w3)

w4=interactive(getvalue, x=ipw.widgets.Checkbox(
    value=False,
    description='Plot integrated accumulation',
    disabled=False
))




In [3]:
def ice_flow_model_01(accu_plot, accu, accu_slope, num_grid): 
    
    global fig1, ax11, ax21, ax22, ax23
    fig1 = plt.figure()
    gs = fig1.add_gridspec(3, 2)

    # create sub plots as grid
    ax11 = fig1.add_subplot(gs[:, 0])
    ax21 = fig1.add_subplot(gs[0, 1])
    ax22 = fig1.add_subplot(gs[1, 1])
    ax23 = fig1.add_subplot(gs[2, 1])

    fig1.set_size_inches(10, 4, forward=True)

    A_0 = 1e-24
    rho_i = 910
    g = 9.81
    n = 3
    C = 2.0* A_0 * (rho_i*g)**n/(n+1)
    
    tmax=500

    x=np.linspace(0, 1000, int(num_grid))
    bed=-x/50
    h=100 - np.power(x/20,2)
    i_0 = np.where( h< 0)
    h[i_0]=0
    h=h.astype('float64')
    s=bed+h
    accu_array=(accu-x*accu_slope)/(365*24*60*60)
    Int_accu = (accu*x-1/2*x*x*accu_slope)/(365*24*60*60)

    # Data for plotting
    num=50*len(x)*len(x)
    t = np.linspace(0, tmax*(365*24*60*60), num)
    L=0*t

    delta_t_init=t[2]-t[1]
    
    delta_t=t[2]-t[1]
    delta_x=x[2]-x[1]
    k=0
    
    while (t[k]<tmax*(365*24*60*60) and k<num-1):
        s=bed+h
        ds_dx_p=(s[2:]-s[1:-1])/delta_x   # 1.5 to end-0.5
        ds_dx_m=(s[1:-1]-s[:-2])/delta_x  # 0.5 to end-1.5

        h_np1=np.power(h,(n+1))

        q_p=-(h_np1[2:]   + h_np1[1:-1])/2.0 * np.power((np.abs(ds_dx_p)),(n-1.0))*ds_dx_p
        q_m=-(h_np1[1:-1] + h_np1[:-2] )/2.0 * np.power((np.abs(ds_dx_m)),(n-1.0))*ds_dx_m

        dq_dx=q_p-q_m  # 1.0 to end-1.0

        q=0*h
        q[1:-1] = C*(q_p+q_m)/2.0
        u=q/(np.abs(h)+1e-6)

        h[1:-1]=h[1:-1] - C*delta_t/delta_x*dq_dx + accu_array[1:-1]*delta_t

        h[0] = h[1] + bed[1] - bed[0]
        h[-1] = h[-2] + bed[-2] - bed[-1]
        i_0 = np.where( h < 0)
        h=np.where(h <= 0, 0, h)

        u=np.where(h < 0, 0, u)
        u[:-1]=np.where(h[1:] <= 0, 0, u[:-1])
        i_0=np.nonzero(h[:-1]) 
        #u[i_0]=0
        #u[i_0[0]-1]=0
        L[k]=x[np.argmax(i_0)]
        h[np.argmax(i_0)+1:]=0 # just in case, setting to zero what should be zero

        if k==0:
            ax21.plot(x/1000, accu_array*(365*24*60*60), color='black')
            ax21.set(ylabel='accu [m/yr]', xlim=(0,max(x/1000)), 
                    ylim=(-accu*1.2,accu*1.2))
            ax21.fill_between(x/1000,accu_array*(365*24*60*60),0, where=(accu_array > 0),color= "blue",alpha= 0.2, label='Snowfall')
            ax21.fill_between(x/1000,accu_array*(365*24*60*60),0, where=(accu_array < 0),color= "red",alpha= 0.2, label='Melting')
                
            if accu_plot:

                ax21b = ax21.twinx()
                ax21b.plot(x/1000, Int_accu, color='r')
                ax21b.set(xlim=(0,max(x/1000)), ylim=(0,max(Int_accu*1.2)))
                
                ax21b.set_ylabel('Integrated accu [m$^2$/s]', color='r')
                ax21b.tick_params(axis='y', colors='r')
            else:
                ax21.legend()
                ax21.grid()
                        
            ax21.set(title='t=%i years' %round(t[k]/(365*24*60*60),0))
            ax21.set_yticks((-accu,0,accu))
            ax11.grid()
        elif k % 10000==0:
            
            ax21.set(title='t=%i years' %round(t[k]/(365*24*60*60),0))

            ax22.clear()
            ax22.plot(x/1000, bed, color='black')
            ax22.fill_between(x/1000,bed,-1000,color= "gray",alpha= 0.2)

            ax22.plot(x/1000, bed+h)
            ax22.set(ylabel='z [m]', 
                   xlim=(0,max(x/1000)), ylim=(-20,0.01+max(bed+h)*1.25))
            
            ax23.clear()
            ax23.plot( (x+delta_x)/1000, u*(365*24*60*60))
            ax23.set(xlabel='x [km]', ylabel='u [m/yr]',
                   xlim=(0,max(x/1000)))
            
            #ax4 = plt.subplot(1, 3, 3)
            ax11.plot(t[:k]/(365*24*60*60), L[:k]/1000, color='black')
            #ax4.plot(t[k]/(365*24*60*60), L[k]/1000, 'o', markersize=12)
            ax11.set(xlabel='t [yrs]', ylabel='L [km]', xlim=(0, tmax), ylim=(0, max(x/1000))) 

            ax21.xaxis.set_tick_params(labelbottom=False)
            ax22.xaxis.set_tick_params(labelbottom=False)
            
            display(plt.gcf())
            
            clear_output(wait=True)
            
        k=k+1
        
    ax21.set(title='t=%i years' %round(t[k]/(365*24*60*60),0))
    
    ax22.clear()
    ax22.plot(x/1000, bed, color='black')
    ax22.fill_between(
        x/1000, 
        bed,
        -1000,
        color= "gray",
        alpha= 0.2)
    ax22.set()

    ax22.plot(x/1000, bed+h)
    ax22.set(ylabel='z [m]', 
           xlim=(0,max(x/1000)), ylim=(-20,0.01+max(bed+h)*1.25)) 


    ax23.clear()
    ax23.plot((x+delta_x)/1000, u*(365*24*60*60))
    ax23.set(xlabel='x [km]', ylabel='u [m/yr]',
           xlim=(0,max(x/1000)))

    #ax4 = plt.subplot(1, 3, 3)
    ax11.plot(t[:k]/(365*24*60*60), L[:k]/1000, color='black')
    #ax4.plot(t[k]/(365*24*60*60), L[k]/1000, 'o', markersize=12)
    ax11.set(xlabel='t [yrs]', ylabel='L [km]', xlim=(0, tmax), ylim=(0, max(x/1000))) 

    ax21.xaxis.set_tick_params(labelbottom=False)
    ax22.xaxis.set_tick_params(labelbottom=False)
    
    
    display(plt.gcf())
    clear_output(wait=True)    
    return h

# Part 1: Mass balance practical - time-dependent evolution

In [4]:
#left_box  = ipw.VBox([w1, w2, w3, w4])
#right_box = ipw.VBox([w2, w6, w7, w8])
#ipw.HBox([left_box,right_box])

left_box  = ipw.VBox([w1])
center_box = ipw.VBox([w2])
right_box = ipw.VBox([w3])
rightR_box = ipw.VBox([w4])
ipw.HBox([left_box, center_box,right_box,rightR_box])

In [5]:
# Loading the input parameters defined by the user
button = ipw.Button(description="Run the model")
output = ipw.Output()
display(button, output)

global iter, h_final, fig1, ax11, ax21, ax22, ax23
iter=0
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        accu=w1.result
        accu_slope=w2.result
        num_grid=w3.result
        accu_plot=w4.result
        
        global iter, h_final
        iter=iter+1 
        
        h_final=ice_flow_model_01(accu_plot,accu, accu_slope,num_grid)
        
button.on_click(on_button_clicked)

Button(description='Run the model', style=ButtonStyle())

Output()

Welcome! In this practical you will run a real <b>land-terminating</b> ice sheet model that solves for ice sheet mass balance and velocity. 

## What you will learn
<ul>
<li> Principles of ice sheet and glacier mass balance
<li> Creating plots in Excel, incl. fitting data with a trendline. (if you follow the full instructions below.)
</ul>

In the model below, the surface mass balance (accumulation and ablation) is given by $$\dot a=(\dot a_0 - a_1 \times x).$$ You can specify $\dot a_0$, $a_1$ and the number of grid points used in the model (less grid points means the model runs faster).

## Instructions for mass-balance class-room exercise
<ol>
<li> Run the model into steady state for 5 different values of $a_0$ between 1 m year$^{-1}$ and 5.0 m year$^{-1}$. Note the values of $a_0$, the value of $a_1$, and the final length of the glacier for each of your runs.
<li> Run the model into steady state for 5 different values of $a_1$ between 0.1 year$^{-1}$ and 1.0 year$^{-1}$. Note the value of $a_0$, the values of $a_1$, and the final length of the glacier for each of your runs.
<li> Create a scatter graph with $a_0$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Add a lienar trendline and note the equation of the trendline.
<li> Create a scatter graph with $a_1$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Add a linear trendline and note the equation of the trendline. 
<li> Would there have been a faster way to determine these relationships?
</ol>